# A guide to produce analysis-ready data

## 1. Load Python modules

In [ ]:
# MODULE                                # DESCRIPTION
import matplotlib.colors as colors      # create visualizations
import matplotlib.image as mpimg        # create visualizations
import matplotlib.pyplot as plt         # create visualizations
from termcolor import colored           # prints colored text
from zipfile import ZipFile             # zip file manipulation
from os.path import join                # data access in file manager  
from glob import iglob                  # data access in file manager
import pandas as pd                     # data analysis and manipulation
import numpy as np                      # scientific computing
import subprocess                       # external calls to system
import snappy                           # SNAP python interface
import jpy                              # Python-Java bridge

# Change module setting
pd.options.display.max_colwidth = 80    # Longer text in pd.df

## 2. Download Sentinel-1 data 

In [ ]:
from sentinelsat import SentinelAPI

user = 'username' 
password = 'password' 
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [ ]:
import geopandas as gpd
import folium 

AOI = gpd.read_file('AOI_name.shp')

m = folium.Map([lat, lon], zoom_start=12)
folium.GeoJson(AOI).add_to(m)
m

In [ ]:
from shapely.geometry import MultiPolygon, Polygon

footprint = None
for i in AOI['geometry']:
    footprint = i

In [ ]:
products = api.query(footprint,
                     date = ('20210601', '20210605'),
                     platformname = 'Sentinel-1',
                     producttype = 'GRD',
                     )

In [ ]:
products_gdf = api.to_geodataframe(products)
products_gdf

In [ ]:
api.download("f0610c38-1845-4bdd-a0b0-626658f30c01")

## 3. Sentinel-1 Preprocessing: snappy
#### User-defined function

In [ ]:
def output_view(product, band, min_value_VV, max_value_VV, min_value_VH, max_value_VH):
    '''
    Creates visualization of processed Sentinel-1 SAR data
    
    Keyword arguments:
    product       -- snappy GPF product --> input Sentinel-1 product 
    band          -- List --> product's band to be visualized
    min_value_VV  -- int --> min value for color strech in VV band
    max_value_VV  -- int --> max value for color strech in VV band
    min_value_VH  -- int --> max value for color strech in VH band
    max_value_VV  -- int --> max value for color strech in VH band
    '''
    band_data_list = []
    
    for i in band:   
        band = product.getBand(i)
        w = band.getRasterWidth()
        h = band.getRasterHeight()
        band_data = np.zeros(w * h, np.float32)
        band.readPixels(0, 0, w, h, band_data)
        band_data.shape = h, w
        band_data_list.append(band_data)
    
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(16,16))
    ax1.imshow(band_data_list[0], cmap='gray', vmin=min_value_VV , vmax=max_value_VV)
    ax1.set_title(output_bands[0])
    ax2.imshow(band_data_list[1], cmap='gray', vmin=min_value_VH , vmax=max_value_VH)
    ax2.set_title(output_bands[1])
    
    for ax in fig.get_axes():
        ax.label_outer()


The easiest way to know which operators are available in snappy is to call gpt -h from command line, which will output the list. If you want to access the documentation of a specific operator, use gpt -h *Operator*

In [ ]:
# Call gpt -h from command line 
print(subprocess.Popen(['gpt','-h', 'Subset'], stdout=subprocess.PIPE, universal_newlines=True).communicate()[0])


<center><img src=Processing_S_1_graph.png width="1400"/></center>


## 3.1 Read product

We start the analysis by setting the folder where the files we want to processed are located. Next, one of the files wil be used as input for this exercise and will be imported with `snappy`. In addition, a quicklook availalbe in the origianl data folder is displayed. 

In [ ]:
# Set target folder and extract metadata
product_path = "/Users/..../original_data"
input_S1_files = sorted(list(iglob(join(product_path, '**', '*S1*.zip'), recursive=True)))

name, sensing_mode, product_type, polarization, height, width, band_names = ([] for i in range(7))

for i in input_S1_files:
    sensing_mode.append(i.split("_")[3])
    product_type.append(i.split("_")[4])
    polarization.append(i.split("_")[-6])
    # Read with snappy
    s1_read = snappy.ProductIO.readProduct(i)  
    name.append(s1_read.getName())
    height.append(s1_read.getSceneRasterHeight())
    width.append(s1_read.getSceneRasterWidth())
    band_names.append(s1_read.getBandNames())

df_s1_read = pd.DataFrame({'Name': name, 'Sensing Mode': sensing_mode,'Product Type': product_type, 'Polarization': polarization ,'Height': height, 'Width': width, 'Available Bands': band_names})
display(df_s1_read)

# Display quicklook - First image
with ZipFile(input_S1_files[0], 'r') as qck_look:
    qck_look = qck_look.open(name[0] + '.SAFE/preview/quick-look.png')
    img = mpimg.imread(qck_look, format='PNG')
    plt.figure(figsize = (15,15))
    plt.title('Quicklook visualiton - '+ name[0] + '\n')
    plt.axis('off')
    plt.imshow(img);

## 3.2 Subset

Once the product is read, we continue our processing chain by creating a subset of the Sentinel-1 product. It is recommened to run this step to reduce processing time. To define the Area Of Interest (AOI), we are defining pixel coordinates, taking as reference the upper right corner of the raster

In [ ]:
# upper right corner
x, y, width, height = 12000, 8000, 5500, 5500

# Subset Operator - snappy
parameters = snappy.HashMap() 
parameters.put('copyMetadata', True)
parameters.put('region', "%s,%s,%s,%s" % (x, y, width, height))
subset = snappy.GPF.createProduct('Subset', parameters, s1_read)
print(list(subset.getBandNames()))

# Plot subset (follow VV - VH order)
output_bands = ['Amplitude_VV', 'Amplitude_VH']
output_view(subset, output_bands, 41, 286, 20, 160)

## 3.3 Apply Orbit File

The orbit state vectors provided in the metadata of a SAR product are generally not accurate and can be refined with the precise orbit files which are available days-to-weeks after the generation of the product. The orbit file provides accurate satellite position and velocity information. Based on this information, the orbit state vectors in the abstract metadata of the product are updated.

In [ ]:
# Apply Orbit File Operator - snappy
parameters = snappy.HashMap()
parameters.put('Apply-Orbit-File', True)
apply_orbit = snappy.GPF.createProduct('Apply-Orbit-File', parameters, subset)
print(colored('Orbit updated succesfully', 'green'))

## 3.4 Thermal Noise Removal

Thermal noise in SAR imagery is the background energy that is generated by the receiver itself. It skews the radar reflectivity towards higher values and hampers the precision of radar reflectivity estimates. Level-1 products provide a noise LUT for each measurement dataset, provided in linear power, which can be used to remove the noise from the product.

In [ ]:
# Thermal Noise Removal Operator - snappy
parameters = snappy.HashMap()
parameters.put('removeThermalNoise', True)
thermal_noise = snappy.GPF.createProduct('ThermalNoiseRemoval', parameters, apply_orbit)

# Plot Thermal Noise Removal (follow VV - VH order)
output_bands = ['Intensity_VV', 'Intensity_VH']
output_view(thermal_noise, output_bands, 0.02, 99376.52, 0.27, 18471.83)

## 3.5 Radiometric Calibration

The objective of SAR calibration is to provide imagery in which the pixel values can be directly related to the radar backscatter of the scene. Though uncalibrated SAR imagery is sufficient for qualitative use, calibrated SAR images are essential to quantitative use of SAR data. Typical SAR data processing, which produces Level-1 images, does not include radiometric corrections and significant radiometric bias remains. The radiometric correction is necessary for the pixel values to truly represent the radar backscatter of the reflecting surface and therefore for comparison of SAR images acquired with different sensors, or acquired from the same sensor but at different times, in different modes, or processed by different processors.

In [ ]:
# Calibration Operator - snappy
parameters = snappy.HashMap()
parameters.put('outputSigmaBand', True)
parameters.put('sourceBands', 'Intensity_VH,Intensity_VV')
parameters.put('selectedPolarisations', 'VH,VV')
parameters.put('outputImageScaleInDb', False)
calibrated = snappy.GPF.createProduct("Calibration", parameters, thermal_noise)

# Plot Calibration (follor VV - VH order)
output_bands = ['Sigma0_VV', 'Sigma0_VH'] 
output_view(calibrated, output_bands, 0.00, 0.28, 0.00, 0.05)

## 3.6 Speckle Filtering

SAR images have inherent salt and pepper like texturing called speckles which degrade the quality of the image and make interpretation of features more difficult. Speckles are caused by random constructive and destructive interference of the de-phased but coherent return waves scattered by the elementary scatters within each resolution cell. Speckle noise reduction can be applied either by spatial filtering or multilook processing.

In [ ]:
# Speckle Filtering Operator - snappy
parameters = snappy.HashMap()
parameters.put('filter', 'Lee')
parameters.put('filterSizeX', 5)
parameters.put('filterSizeY', 5)
speckle = snappy.GPF.createProduct('Speckle-Filter', parameters, calibrated)

# Plot speckle filter (follow VV - VH order)
output_bands = ['Sigma0_VV', 'Sigma0_VH'] 
output_view(speckle, output_bands, 0.00, 0.28, 0.00, 0.05)

## 3.7 Terrain Correction

Due to topographical variations of a scene and the tilt of the satellite sensor, distances can be distorted in the SAR images. Image data not directly at the sensor’s Nadir location will have some distortion. Terrain corrections are intended to compensate for these distortions so that the geometric representation of the image will be as close as possible to the real world.

In [ ]:
#Get proj information from .xml file (graph) that includes Range Doppler Terrain Correction with UTM/Automtic parameter
proj = '''PROJCS["UTM Zone 35 / World Geodetic System 1984", 
  GEOGCS["World Geodetic System 1984", 
    DATUM["World Geodetic System 1984", 
      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], 
      AUTHORITY["EPSG","6326"]], 
    PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], 
    UNIT["degree", 0.017453292519943295], 
    AXIS["Geodetic longitude", EAST], 
    AXIS["Geodetic latitude", NORTH]], 
  PROJECTION["Transverse_Mercator"], 
  PARAMETER["central_meridian", 27.0], 
  PARAMETER["latitude_of_origin", 0.0], 
  PARAMETER["scale_factor", 0.9996], 
  PARAMETER["false_easting", 500000.0], 
  PARAMETER["false_northing", 0.0], 
  UNIT["m", 1.0], 
  AXIS["Easting", EAST], 
  AXIS["Northing", NORTH]]''' 

# Terrain-Correction Operator - snappy
parameters = snappy.HashMap()
parameters.put('demName', 'SRTM 3Sec')
parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
parameters.put('pixelSpacingInMeter', 10.0)
parameters.put('mapProjection', proj)  
parameters.put('nodataValueAtSea', False) # do not mask areas without elevation (WATER AREAS!)
parameters.put('saveSelectedSourceBand', True)
terrain_correction = snappy.GPF.createProduct('Terrain-Correction', parameters, speckle)

# Plot terrain correction (follow VV - VH order)
output_bands = ['Sigma0_VV', 'Sigma0_VH'] # in this step Amplitude bands are lost?
output_view(terrain_correction, output_bands,0.00, 0.49, 0.00, 0.04)

## 3.8 Write

Once we have completed all preprocessing steps we can write our SAR product to file. In this occasion we will chooose the GeoTIFF format.

In [ ]:
# Set output path and name
outpath_name = '/Users/..../output_data/20200321_Orb_Thm_Cal_Spk_TC'

# Write Operator - snappy
snappy.ProductIO.writeProduct(terrain_correction, outpath_name, 'GeoTIFF')
print(colored('Product succesfully saved in:', 'green'), outpath_name)